In [1]:
import os
import re
import fnmatch
import pandas as pd
import numpy as np
import warnings
import split_extract
from multiprocessing import Pool
from tqdm import tqdm
from params import dresden_images_root, images_db_path, patch_span, \
        patch_num, robust_root, robust_db_path

In [3]:
warnings.filterwarnings("ignore")

if not os.path.exists(robust_root):
    os.makedirs(robust_root)

In [4]:
images_db = np.load(images_db_path, allow_pickle=True).item()

model_list = np.unique(images_db['brand_model'])
test_model_list = list(filter(lambda model: re.search(r'^((?!Canon).)*$', model), model_list))
img_list = images_db['path']
test_img_list = list(filter(lambda img: re.search(r'^((?!Canon).)*$', img), img_list))

In [6]:
for model in test_model_list:
    test_list = fnmatch.filter(test_img_list, model + '*')
    print("{} in test set: {}.".format(model, len(test_list)))

Agfa_DC-504 in test set: 169.
Agfa_DC-733s in test set: 281.
Agfa_DC-830i in test set: 363.
Agfa_Sensor505-x in test set: 172.
Agfa_Sensor530s in test set: 372.


In [ ]:
# files_labels = dict(zip(images_db['path'], images_db['brand_model']))
# test_labels = []

# for path in test_img_list:
#     test_labels += [files_labels[path]]

In [ ]:
for img_brand_model, img_path in \
    tqdm(zip(test_model_list, test_img_list)):
        test_list += [{'data_set':'test',
                       'img_path':img_path,
                       'img_brand_model':img_brand_model,
                       'patch_span':patch_span,
                       'patch_num':patch_num,
                       'patch_root': robust_root,
                       'img_root': dresden_images_root
                       }]

# num_processes = 12
num_processes = 4
pool = Pool(processes=num_processes)

test_paths = pool.map(split_extract.extract, test_list)
print('Completed.')

2336it [00:00, 519121.23it/s]
